# **Sentimen Analisis Review APK Play Store**



# **Import Library**

In [13]:
import pandas as pd  # Pandas untuk manipulasi dan analisis data
pd.options.mode.chained_assignment = None  # Menonaktifkan peringatan chaining
import numpy as np  # NumPy untuk komputasi numerik
seed = 0
np.random.seed(seed)  # Mengatur seed untuk reproduktibilitas
import matplotlib.pyplot as plt  # Matplotlib untuk visualisasi data
import seaborn as sns  # Seaborn untuk visualisasi data statistik, mengatur gaya visualisasi

import datetime as dt  # Manipulasi data waktu dan tanggal
import re  # Modul untuk bekerja dengan ekspresi reguler
import string  # Berisi konstanta string, seperti tanda baca
from nltk.tokenize import word_tokenize  # Tokenisasi teks
from nltk.corpus import stopwords  # Daftar kata-kata berhenti dalam teks

!pip install sastrawi
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory  # Stemming (penghilangan imbuhan kata) dalam bahasa Indonesia
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory  # Menghapus kata-kata berhenti dalam bahasa Indonesia

from wordcloud import WordCloud  # Membuat visualisasi berbentuk awan kata (word cloud) dari teks

In [3]:
import nltk  # Import pustaka NLTK (Natural Language Toolkit).
nltk.download('punkt')  # Mengunduh dataset yang diperlukan untuk tokenisasi teks.
nltk.download('stopwords')  # Mengunduh dataset yang berisi daftar kata-kata berhenti (stop words) dalam berbagai bahasa.

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

# Loading Dataset

In [14]:
app_reviews_df = pd.read_csv('mobile_legend_fiks.csv', on_bad_lines='skip')

In [15]:
app_reviews_df.shape
app_reviews_df.head()

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion
0,9bd15e3b-7ae1-4827-af1c-f80b9aea4fa2,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,tiap main rank pasti aja ada player beban/male...,1,22705,1.9.65.10602,2025-03-28 15:42:15,NaN,NaN,1.9.65.10602
1,04893854-ffc4-4a5a-bd53-18dda4879354,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,jika setiap update game rank nya selalu saja m...,2,936,1.9.65.10602,2025-03-30 16:27:41,NaN,NaN,1.9.65.10602
2,32d316bf-a701-4017-9746-8c3b71dc0892,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"Bintang 3 aja moonton, grafik sudah oke, efek ...",3,1757,1.9.65.10602,2025-03-29 18:36:17,"Halo Kak,\nKami berkomitmen untuk menciptakan ...",2025-01-04 04:00:00,1.9.65.10602
3,b541e597-ea22-4560-a4c2-a46643e719e1,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,kemarin install lagi ini game dengan harapan s...,1,993,1.9.65.10602,2025-03-28 23:50:06,NaN,NaN,1.9.65.10602
4,82dd9f50-2f90-4efe-a385-88f80e456f14,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,Game ini bagus sangat!! Tetapi tolong sekali h...,5,3368,1.9.65.10602,2025-03-21 06:07:28,NaN,NaN,1.9.65.10602


In [16]:
# Hanya ambil data yang kolom 'content' dan 'score'-nya tidak kosong
clean_df = app_reviews_df[app_reviews_df['content'].notna() & app_reviews_df['score'].notna()]

# Reset index agar rapi
clean_df = clean_df.reset_index(drop=True)

# Cek hasilnya
clean_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 126000 entries, 0 to 125999
Data columns (total 11 columns):
 #   Column                Non-Null Count   Dtype 
---  ------                --------------   ----- 
 0   reviewId              126000 non-null  object
 1   userName              126000 non-null  object
 2   userImage             126000 non-null  object
 3   content               126000 non-null  object
 4   score                 126000 non-null  int64 
 5   thumbsUpCount         126000 non-null  int64 
 6   reviewCreatedVersion  103502 non-null  object
 7   at                    126000 non-null  object
 8   replyContent          5412 non-null    object
 9   repliedAt             5412 non-null    object
 10  appVersion            103502 non-null  object
dtypes: int64(2), object(9)
memory usage: 10.6+ MB


In [23]:
# Salin DataFrame utama (opsional untuk keamanan)
df_clean = app_reviews_df.copy()

# DROP kolom dengan mayoritas missing value
df_clean = df_clean.drop(columns=['replyContent', 'repliedAt'])

# DROP baris yang missing pada kolom penting jika ingin memakai reviewCreatedVersion dan appVersion
df_clean = df_clean.dropna(subset=['reviewCreatedVersion', 'appVersion'])

# Drop kolom yang banyak missing dan tidak digunakan
clean_df.drop(columns=["replyContent", "repliedAt", "reviewCreatedVersion", "appVersion"], inplace=True)

# Cek kembali info setelah pembersihan
df_clean.info()

# Tampilkan jumlah data akhir
print(f"\nJumlah data setelah penanganan missing value: {len(df_clean)}")


<class 'pandas.core.frame.DataFrame'>
Index: 103502 entries, 0 to 125999
Data columns (total 9 columns):
 #   Column                Non-Null Count   Dtype 
---  ------                --------------   ----- 
 0   reviewId              103502 non-null  object
 1   userName              103502 non-null  object
 2   userImage             103502 non-null  object
 3   content               103502 non-null  object
 4   score                 103502 non-null  int64 
 5   thumbsUpCount         103502 non-null  int64 
 6   reviewCreatedVersion  103502 non-null  object
 7   at                    103502 non-null  object
 8   appVersion            103502 non-null  object
dtypes: int64(2), object(7)
memory usage: 7.9+ MB

Jumlah data setelah penanganan missing value: 103502


In [24]:
clean_df.isnull().sum()

,0
reviewId,0
userName,0
userImage,0
content,0
score,0
thumbsUpCount,0
at,0


In [25]:
# Menghapus baris duplikat dari DataFrame clean_df
clean_df = clean_df.drop_duplicates()

# Menghitung jumlah baris dan kolom dalam DataFrame clean_df setelah menghapus duplikat
jumlah_ulasan_setelah_hapus_duplikat, jumlah_kolom_setelah_hapus_duplikat = clean_df.shape

In [ ]:
clean_df.isnull().sum()

,0
reviewId,0
userName,0
userImage,0
content,0
score,0
thumbsUpCount,0
at,0


##Preprocessing

In [26]:
import re
import string
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

nltk.download('punkt')
nltk.download('stopwords')

# Fungsi normalisasi slang
def replace_slang(text, slang_dict):
    words = re.findall(r'\b\w+\b|[^\w\s]', text.lower())
    cleaned_words = [slang_dict.get(word, word) for word in words]
    return ' '.join(cleaned_words)

# Membersihkan noise dari teks
def cleaningText(text):
    text = re.sub(r'@[A-Za-z0-9_]+', '', text)     # mention
    text = re.sub(r'#[A-Za-z0-9_]+', '', text)     # hashtag
    text = re.sub(r'RT[\s]+', '', text)            # retweet
    text = re.sub(r"http\S+", '', text)            # link
    text = re.sub(r'[0-9]+', '', text)             # angka
    text = re.sub(r'[^\w\s]', '', text)            # simbol non-alfanumerik
    text = text.replace('\n', ' ').strip()
    return text

# Huruf kecil semua
def casefoldingText(text):
    return text.lower()

# Tokenisasi
def tokenizingText(text):
    return word_tokenize(text)

# Filtering (stopword removal + tambahan stopword informal)
def filteringText(tokens):
    stop_words = set(stopwords.words('indonesian')).union(set(stopwords.words('english')))
    tambahan = {'iya','yaa','gak','nya','na','sih','ku','di','ga','ya','gaa',
                'loh','lah','kah','woi','woii','woy','lo','aja','tuh','nih','mah'}
    stop_words.update(tambahan)
    return [word for word in tokens if word not in stop_words]

# Stemming (ubah dari list token)
def stemmingText(tokens):
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()
    return [stemmer.stem(word) for word in tokens]

# Gabungkan kembali ke kalimat
def toSentence(tokens):
    return ' '.join(tokens)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [27]:
slangwords = {
    'id': 'identitas ','dibiarin': 'dibiarkan','yg': 'yang','tolol': 'bodoh', "@":'di','moba':'mobile legend',' kaga':'tidak','samsek': 'sama sekalih',
    'ijo': 'hijau','apk':'aplikasi','lu':'kamu','mm':'main','pisahin':'pisahkan','tiap':'setiap','males':'malas','/':'atau','&':'dan','lancar²':'lancar',
    'baguss': 'bagus', 'terimakasih': 'terima kasih','mudarat': 'kerugian','Jaringab':'jaringan','ms':'milidetik','bgt pas':'sangat cocok','blm':'belum',
    'capek': 'lelah', 'mulu': 'terus-menerus', 'kek': 'seperti','ff': 'free fire', 'ngabisin': 'menghabiskan','Uda':'sudah','ngeluh':'mengeluh','jelek':'buruk',
    'keseruannya': 'keseruan', 'sia': 'sia-sia', 'tidak merekomendasikan': 'tidak menyarankan', 'ban': 'blokir','bosok':'busuk','toloong ':'tolong','dst':'dan seterusnya',
    'Gemnya':'game-nya','uda':'sudah','skin2':'skin','keren':'hebat','plis':'tolong','benerin':'benarkan','ya':'iya','relog':'masuk kembali','dong':'mohon',
    'nyampe': 'sampai', 'perbaikilah': 'perbaiki', 'meresahkan': 'mengganggu', 'pensi': 'berhenti','eh':'tetapi','doang':'saja','pake':'pakai','Feeder':'pengumpan',
    'lane': 'jalur', 'gimana': 'bagaimana', 'ganggu': 'mengganggu','in':'dalam','TDK':'tidak','gadiubah':'tidak diubah','Sejago':'biar hebat',
    'diawal': 'di awal', 'wifi': 'jaringan', 'mati': 'terputus', 'login': 'masuk','kalo':'kalau','vs':'versus','setim ':'satu tim','dark system ':'di rugikan','pantes':'pantas',
    'ngebug': 'bug', 'lemot': 'lambat', 'ngestuck': 'macet','apa2':'apa-apa','ketat':'teliti',' movement':'pergerakan','fps':'tembak sudut-pertama','lv':'level','agak':'sedikit',
    'ngecrash': 'crash', 'ngaco': 'berantakan', 'ngilangin': 'menghilangkan', 'ngasih': 'memberikan','oi':'halo','tier':'tingkat','pencaharian':'pencarian','Dear':'untuk',
    'jelekk': 'buruk', 'jelek': 'buruk', 'nge-lag': 'lag', 'ngikut': 'mengikuti','Overall':'keseluruhan','player2':'player-player','nabrak':'menabrak','ws ':'kemenangan',
    'kesel': 'kesal', 'sampe': 'sampai', 'sampek': 'sampai',' langkah2 ':'langkah nya','sampe²':'sampai','jls':'jelas','gb':'gigabyte',' tiba²':'tiba-tiba','pengen':'mau',
    'bikin': 'membuat', 'maksa': 'memaksa', 'mending': 'lebih baik', 'gitu': 'seperti itu','ID':'identitas','rese':'usil','enjoy':'santai','patah²':'patah-patah','orang²':'orang-orang',
    'kayak': 'seperti', 'bangett': 'banget', 'pdhl': 'padahal', 'udh': 'sudah', 'udah': 'sudah','g':'tidak','jgn':'jangan','MB':'megabyte','apalahhh':'apalah',
    'btw': 'ngomong-ngomong', 'ni': 'ini', 'dahl': 'padahal', 'dr': 'dari', 'klo': 'kalau','mainin':'memainkan','issue':'hasil akhir','medsos':'media sosial','lamcar':'lancar',
    'kalo': 'kalau', 'lg': 'lagi', 'jg': 'juga', 'dpt': 'dapat', 'tp': 'tapi', 'tmn': 'teman',' By the way':'ngomong-ngomong','dm':'jarak','hehehe':'ketawa','ilang':'hilang',
    'doang': 'saja', 'bgtu': 'begitu', 'cmn': 'cuma', 'cuma': 'hanya', 'bkin': 'membuat','momen²':'momen-momen','keluarin':'keluarkan','character':'karakter','sya':'saya',
    'ml': 'mobile legend', 'skill': 'kemampuan', 'emblem': 'atribut', 'win': 'menang','kembbalikan':'kembalikan','cepet':'cepat','ngalamin':'mengalami','temen':'teman',
    'winstreak': 'menang beruntun','cs': 'layanan pelanggan','jga':'juga','bareng':'bersama','maen':'main','ad':'ada','Ama':'sama','matiin':'mematikan','donk':'dong','aduhh':'gawat',
    'mvp': 'pemain terbaik', 'crash': 'berhenti mendadak', 'mohon': 'harap', 'player': 'pemain','kok':'juga','ngelegtapi':'ngelag tapi','buagus polll':'bagus sekalih','bener bener':'benar-benar',
    'players': 'pemain', 'troll': 'pengganggu','hp':'ponsel','ketang':'lemot','d tengah2':'di tengah-tengah','dah':'sudah','ntr':'sebentar','d kata2in':'dihina','hbs':'habis','turen':'menara',
    'matching': 'penjodohan', 'matchingnya': 'penjodohan pertandingan', 'dark system': 'sistem buruk','lngsung':'langsung','many tq':'terima kasih banyk','brawl':'pertarungan','leg':'lag','Gimana':'bagaimana',
    'delay': 'penundaan', 'team war': 'pertempuran tim','makasih papi':'terimah kasih bapak','temenya':'teman-nya','penalites??':'penalti','Abis':'habis','jatoh':'jatuh','thirdparty':'pihak ketiga','score':'skor',
    'ws': 'menang beruntun', 'tier': 'tingkatan','permainannya':'permainan-nya','nubb':'noob','noob':'pemula','sebenernya':'sebenarnya','mh':'saja','utamain':'utamakan','bettle field':'medan pertarungan','cuma':'cuman',
    'depresi': 'stres', 'buff': 'penambah kekuatan', 'kill': 'mengalahkan', 'toxic': 'kasar','ngasih':'kasih','burik':'buruk','jan':'jangan','bgeni':'begini','nama ny':'nama-nya','berkali2':'berkali-kali',
    'role': 'peran','gaubah': 'tidak diubah', 'mainnya': 'permainannya', 'hp': 'ponsel', 'masuk2': 'masuk-masuk','bertahun2':'bertahun-tahun','apaan':'apa','siih':'mengeluh',
    'dimaksimalkan': 'dioptimalkan', 'pesan': 'notifikasi', 'awlnya': 'awalnya', 'gajelas': 'tidak jelas','turunin':'turunkan','alu':'palu','dlu':'dulu','begronnya tigreal': 'bagaimana cara bermain Tigreal',
    'ngefreeze': 'beku', 'ngechat': 'mengirim pesan', 'ngeleg': 'lag', 'stuck': 'macet','updatan x':'pembaruan-nya','dikit dikit':'sedikit','knp':'kenapa','anjimmmmmm':'memaki','gimanasi':'bagaimana',
    'gw': 'saya', 'gua': 'saya', 'gue': 'saya', 'gk': 'tidak', 'ga': 'tidak', 'gak': 'tidak', 'g': 'tidak','beratek tinggi':'berat tinggi','Gapapa':'tidak apa-apa','kil':'skill','koneksinya':'koneksi-nya',
    'tdk': 'tidak', 'trs': 'terus', 'gbs': 'tidak bisa', 'aj': 'saja', 'aja': 'saja', 'bgt': 'banget','end nya':'akhir-nya','bisaa':'bisa','jirrr':'mengeluh','woi':'halo','pls':'plis','Gw':'saya','dibrnarkan':'dibiarkan','dibanyakin':'dibanyakan',
    'bgd': 'banget', 'bngt': 'banget', 'bngt2': 'banget-banget', 'bae': 'baik', 'parahh': 'parah','nggk':'tidak','gam':'game','gua':'saya','nob':'noob','download':'unduh','suara n music':'suara dan musik','udh on':'sudah aktif',
    'parahhh': 'parah', 'dm': 'diamond', 'match': 'pertandingan', 'relag': 'relog', 'relog': 'masuk ulang','taik':'menghina','nooob':'noob','sdh':'sudah','n':'dan','TAIK':'mengeluh','naikin':'naikan','gtuuuu':'begitu',
    'nggak': 'tidak', 'ngak': 'tidak', 'ngga': 'tidak', 'pvp': 'pertarungan pemain','moontonnnnn':'moonton','gratisA':'gratis','ngeselin':'menyebalkan','pelayer nya':'pemainya','laq parahhh':'lag parah','ni anjim':'memaki','ngawur':'tidak masuk akal',
    'ms': 'milidetik', 'afk': 'tidak aktif', 'op': 'overpowered', 'chat': 'pesan', 'heronya': 'hero-nya','Napa':'kenapa','kepencet':'tertekan','bnyk':'banyak','e':'nya','ndak':'tidak','ngentod':'memaki','lihat2':'lihat-lihat','teman2':'teman-teman','hahahaha':'ketawa',
    'dev': 'pengembang', 'eventnya': 'acara-nya', 'event': 'acara', 'buq': 'bug', 'bugnya': 'bug-nya','php':'pemberi harapan palsu','akhir eh':'akhirnya','dl':'dulu','konesi':'koneksi','setabil':'stabil','monton':'moonton',
    'spin': 'putaran', 'ranked': 'peringkat', 'matchmaking': 'pencocokan pertandingan','telkom':'telkomsel','utk':'untuk','dalem':'dalam','warnaa':'warna','mohonn bantuannya':'mohon bantuan-nya','Baguss':'bagus','gamenyaa':'game-nya',
    'battle': 'pertempuran', 'skin': 'kulit karakter', 'loby': 'lobi', 'loading': 'memuat','truss':'terus','min':'admin','sm':'sama','noh':'sana','emg':'memang','kya':'kayak','pad':'pada','histori':'cerita','lgi':'lagi','closs':'close','close':'tutup',
    'scam': 'penipuan', 'cheat': 'curang', 'credit score': 'skor kredit', 'score': 'skor','tunnggin':'tuningkan','pke':'pakai','practice':'praktis','bsa':'bisa','mnding':'mending','hadeh':'tidak jelas','gm':'game',
    'resource': 'sumber daya', 'ping': 'latensi', 'eror': 'error', 'koneksi': 'jaringan','chet':'chat','HEMMM':'mengejek','gg':'hebat','musu':'musuh','diperioritaskan':'diprioritaskan',
    'ngelag': 'koneksi lambat', 'nge lag': 'koneksi lambat', 'ngestuck': 'macet', 'afk': 'tidak aktif', 'troll': 'bermain merugikan tim','setiap x':'setiap-nya','upded':'update','pliis':'plis','sakit x':'sakitnya','lemal':'lemah','leg nua':'lag-nya','ben':'ban','mnarik':'menarik',
    'ngetroll': 'bermain merugikan tim', 'dark system': 'sistem tidak adil', 'drak sistem': 'sistem tidak adil', 'matchmaking': 'pencocokan pertandingan','mvp': 'pemain terbaik','candu':'kecanduan','gnti':'ganti','org':'orang','lelet':'lambat','apdate':'update','Tros':'terus',
    'auto lose': 'pasti kalah','bug': 'kesalahan sistem', 'bugnya': 'kesalahan sistem','ketulis':'tertulis','tuker':'tukar','bag':'bug','keren keren':'keren-keren','bner':'benar','cpe':'capek','capek':'lelah',
    'ulti': 'jurus pamungkas', 'dapet tim tolol': 'mendapat tim buruk', 'tim tolol': 'tim buruk', 'musuh jago': 'lawan hebat', 'server': 'peladen','ngsih':'kasih','ngegendongnya': 'menggendong','nyeseli':'menyesalkan','napa':'kenapa','yp':'yang','Tod':'memaki',
    'serfer': 'peladen', 'monton': 'moonton', 'moonton': 'pengembang', 'speed mode': 'mode kecepatan','ml': 'mobile legend','masalh':'masalah','geme':'game','loding':'loading','Astaga':'ya ampun','y':'iya','ilang':'hilang','gx':'tidak',
    'mobile lagend': 'mobile legend','mobile': 'mobile legend','auto lose': 'pasti kalah', 'winrate': 'persentase kemenangan','bugnya': 'kesalahan sistem','d buka lg':'dibuka lagi','trun':'turun','ketas':'keatas','basmi':'habisi','WR.WB':'warahmatullahi wabarakatuh',
    'mobile lagend': 'mobile legend', 'mobile lagand': 'mobile legends', 'kntl': 'penis', 'kontol': 'penis', 'anj': 'anjing', 'bangkek': 'bangkai','plissss':'plis','gag':'tidak ada','br':'baru','nyatan':'santai','lbh':'lebih',
    'ancvkkk': 'memaki', 'pensi': 'pensiun', 'pensiun aja': 'berhenti bermain', 'gblok': 'bodoh', 'tolol': 'bodoh', 'bego': 'bodoh', 'kayak bot': 'sangat buruk','kebuka2':'terbuka','wjar':'wajar','bnyar':'banyak','terdlu':'terdahulu',
    'bot': 'kecerdasan buatan', 'cheat': 'curang', 'custom mode': 'mode khusus','op': 'terlalu kuat','lag': 'koneksi lambat','dc': 'terputus', 'lemot': 'lambat','kcanduan':'kecanduan','sia2':'sia-sia','sring':'sering',
    'eventnya': 'acara-nya', 'spin': 'putaran', 'ranked': 'peringkat', 'battle': 'pertempuran', 'loby': 'lobi', 'scam': 'penipuan', 'score': 'skor', 'ping': 'latensi','ngerti':'mengerti','Lejen':'legend','ktmu':'ketemu','kwan':'kawan',
    'eror': 'error', 'dev': 'pengembang','Gak' : 'tidak', 'garena': 'karena', 'nih': 'ini','Min' : 'admin', 'y': 'iya', 'ngadepin': 'menghadapi','inir':'ini','iini':'ini','mobilegen':'mobile legend','BKN':'bukan','pd':'pada','woyy':'hey',
    'nie':'ini','sma':'sama','mmr':'memori','kayak':'sangat','emang':'memang','trims': 'terima kasih','mksh': 'terima kasih','nih': 'ini','mlbb' : 'mobile legend bang bang','perbaikiterkadang' : 'perbaiki terkadang',
    'ajabisa': 'aja bisa', 'dapet' : 'dapat', 'sumpahh': 'sumpah', 'bebanmales':'beban malas','Trimksh':'terima kasih','seneng':'senang','jir':'memaki','hadehhh':'mengeluh','tlongg':'tolong','diperbaikii':'diperbaiki',

}

def fix_slangwords(text):
    words = text.split()
    fixed_words = []

    for word in words:
        fixed_words.append(slangwords.get(word.lower(), word))

    return ' '.join(fixed_words)


In [28]:
clean_df

,reviewId,userName,userImage,content,score,thumbsUpCount,at
0,9bd15e3b-7ae1-4827-af1c-f80b9aea4fa2,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,tiap main rank pasti aja ada player beban/male...,1,22705,2025-03-28 15:42:15
1,04893854-ffc4-4a5a-bd53-18dda4879354,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,jika setiap update game rank nya selalu saja m...,2,936,2025-03-30 16:27:41
2,32d316bf-a701-4017-9746-8c3b71dc0892,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"Bintang 3 aja moonton, grafik sudah oke, efek ...",3,1757,2025-03-29 18:36:17
3,b541e597-ea22-4560-a4c2-a46643e719e1,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,kemarin install lagi ini game dengan harapan s...,1,993,2025-03-28 23:50:06
4,82dd9f50-2f90-4efe-a385-88f80e456f14,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,Game ini bagus sangat!! Tetapi tolong sekali h...,5,3368,2025-03-21 06:07:28
...,...,...,...,...,...,...,...
125995,74ec577d-9781-4f2b-b958-f0f789bcf3c3,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,Dear monton. Ini knpa sya login sudh mau rank ...,1,0,2019-07-04 11:20:45
125996,dae9e2ee-595e-419a-bd59-96a038bc22f2,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,Tolong untuk developer agar diperbaiki lagi sa...,3,0,2024-02-01 08:15:06
125997,417f0759-c9db-4b27-8a6d-30be3c450b31,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,Saya kecewa berat dengan game ini soalnya jari...,5,0,2020-05-16 01:32:26
125998,2de9fe47-bdf1-403d-a4bf-47c78e623987,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,tambah update entah kenapa semakin terasa pata...,2,0,2023-12-04 05:37:47


In [29]:
import nltk
from nltk.corpus import stopwords # Import stopwords from nltk.corpus

# nltk.download('punkt')
nltk.download('stopwords')
nltk.download('punkt_tab') # Download the missing 'punkt_tab' resource

# Membersihkan teks dan menyimpannya di kolom 'text_clean'
clean_df['text_clean'] = clean_df['content'].apply(cleaningText)

# Mengubah huruf dalam teks menjadi huruf kecil dan menyimpannya di 'text_casefoldingText'
clean_df['text_casefoldingText'] = clean_df['text_clean'].apply(casefoldingText)

# Mengganti kata-kata slang dengan kata-kata standar dan menyimpannya di 'text_slangwords'
clean_df['text_slangwords'] = clean_df['text_casefoldingText'].apply(lambda x: replace_slang(x, slangwords))

# Memecah teks menjadi token (kata-kata) dan menyimpannya di 'text_tokenizingText'
clean_df['text_tokenizingText'] = clean_df['text_slangwords'].apply(tokenizingText)

# Define stop_words here using the imported stopwords
stop_words = set(stopwords.words('indonesian')) # Change 'english' to 'indonesian' for Indonesian stopwords

# Menghapus kata-kata stop (kata-kata umum) dan menyimpannya di 'text_stopword'
clean_df['text_stopword'] = clean_df['text_tokenizingText'].apply(filteringText)

# Menggabungkan token-token menjadi kalimat dan menyimpannya di 'text_akhir'
clean_df['text_akhir'] = clean_df['text_stopword'].apply(toSentence)

# Tampilkan hasil preprocessing
print(clean_df[['content', 'text_akhir']].head())

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


                                             content  \
0  tiap main rank pasti aja ada player beban/male...   
1  jika setiap update game rank nya selalu saja m...   
2  Bintang 3 aja moonton, grafik sudah oke, efek ...   
3  kemarin install lagi ini game dengan harapan s...   
4  Game ini bagus sangat!! Tetapi tolong sekali h...   

                                          text_akhir  
0  main rank pemain beban malas main pemain penga...  
1  update game rank mengurang lelah menyelesaikan...  
2  bintang pengembang grafik oke efek visual kema...  
3  kemarin install game harapan semoga kendala ra...  
4  game bagus tolong hilangkan kesalahan sistem m...  


In [ ]:
!pip install scikit-learn

In [ ]:
print(clean_df['text_akhir'].sample(5))

46609    hadeh habis update masuk game masuk menu tema ...
82849    menyesal season season bagus season game sebel...
92690    gamenya bagus update kredit skor ganti ponsel ...
39933    kesalahan sistem pemain kesini seru game gm ti...
55952    suka lag sinyal bagus tolong mohon pengembang ...
Name: text_akhir, dtype: object


In [ ]:
import numpy as np

non_zero_counts = np.count_nonzero(X_tfidf.toarray(), axis=1)
print(non_zero_counts[:10])  # Cek 10 ulasan pertama

[17 19 22 22 18 27 26 14 25 26]


##Pelabelan

In [30]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

# Fungsi untuk menentukan sentiment berdasarkan skor
def get_sentiment(score):
    if score >= 4:
        return 'positive'
    elif score == 3:
        return 'neutral'  # Ini masih ada, tapi nanti akan dihapus
    else:
        return 'negative'

# Menambahkan kolom 'sentiment' berdasarkan skor
clean_df['sentiment'] = clean_df['score'].apply(get_sentiment)

# Menghapus data dengan kelas 'neutral'
filtered_df = clean_df[clean_df['sentiment'] != 'neutral']

# Menyiapkan fitur (X) dan label (y) untuk pelatihan
X = filtered_df['text_akhir']  # Menggunakan teks yang sudah dibersihkan
y = filtered_df['sentiment']  # Kolom sentiment yang hanya ada 'positive' dan 'negative'

# Mengubah teks menjadi fitur numerik menggunakan TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_features=5000)
X_tfidf = tfidf_vectorizer.fit_transform(X)

# Membagi data menjadi data pelatihan dan pengujian
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.2, random_state=42)

# Menampilkan bentuk data pelatihan dan pengujian
print(f"Bentuk data pelatihan: {X_train.shape}")
print(f"Bentuk data pengujian: {X_test.shape}")


Bentuk data pelatihan: (90640, 5000)
Bentuk data pengujian: (22661, 5000)


##Melatih model

### Random Forest

In [44]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
import time

# Mulai pengukuran waktu
start_time = time.time()

# Buat model Random Forest
model = RandomForestClassifier(n_estimators=100, random_state=42)

# Latih model
model.fit(X_train, y_train)

# Prediksi data uji
y_pred = model.predict(X_test)

# Hitung waktu eksekusi
execution_time = time.time() - start_time

# Evaluasi model
accuracy = accuracy_score(y_test, y_pred)
print(f"Akurasi: {accuracy:.4f}")
print(f"Waktu eksekusi: {execution_time:.2f} detik")
print(classification_report(y_test, y_pred))

Akurasi: 0.8695
Waktu eksekusi: 596.24 detik
              precision    recall  f1-score   support

    negative       0.84      0.91      0.88     15707
    positive       0.91      0.82      0.86     15707

    accuracy                           0.87     31414
   macro avg       0.87      0.87      0.87     31414
weighted avg       0.87      0.87      0.87     31414



Berdasarkan hasil evaluasi dari beberapa algoritma yang diujikan, model Random Forest menunjukkan performa yang paling optimal dalam mengklasifikasikan sentimen,akurasi yang diperoleh masih berada pada angka 0.86%, nilai ini tetap lebih tinggi dibandingkan model-model lain yang telah dicoba seperti Logistic Regression,SVM,dan Stochastic Gradient Descent. Hal ini menunjukkan bahwa Random Forest mampu menangkap pola dalam data lebih baik, terutama dalam konteks kompleksitas dan variasi fitur pada dataset. Dengan potensi tuning lebih lanjut dan perbaikan preprocessing, model ini dapat dikembangkan menjadi lebih akurat dan andal untuk tugas analisis sentimen.

### SVM

In [35]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report, accuracy_score
from imblearn.over_sampling import SMOTE

# === 3. TF-IDF Vectorizer ===
tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_features=20000, ngram_range=(1, 2))
X_tfidf = tfidf_vectorizer.fit_transform(X)

# === 4. SMOTE Balancing ===
sm = SMOTE(random_state=42)
X_res, y_res = sm.fit_resample(X_tfidf, y)

# === 5. Split Data ===
X_train, X_test, y_train, y_test = train_test_split(
    X_res, y_res, test_size=0.2, stratify=y_res, random_state=42
)

# === 6. Grid Search SVM ===
param_grid = {'C': [0.01, 0.1, 0.5, 1, 2, 5, 10]}
grid = GridSearchCV(LinearSVC(max_iter=2000), param_grid, cv=5)
grid.fit(X_train, y_train)

best_model = grid.best_estimator_
y_pred = best_model.predict(X_test)

# === 7. Evaluasi ===
acc = accuracy_score(y_test, y_pred)
print(f"\nBest Parameters: {grid.best_params_}")
print(f"Accuracy: {acc:.4f}")
print(classification_report(y_test, y_pred))



Best Parameters: {'C': 0.1}
Accuracy: 0.8418
              precision    recall  f1-score   support

    negative       0.83      0.86      0.84     15707
    positive       0.85      0.82      0.84     15707

    accuracy                           0.84     31414
   macro avg       0.84      0.84      0.84     31414
weighted avg       0.84      0.84      0.84     31414



## logistic Regression

In [39]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score

# Membuat model Logistic Regression
model = LogisticRegression(max_iter=4000)

# Melatih model
model.fit(X_train, y_train)

# Memprediksi dengan data pengujian
y_pred = model.predict(X_test)

# Evaluasi model
accuracy = accuracy_score(y_test, y_pred)
print(f"Akurasi: {accuracy:.4f}")
print(classification_report(y_test, y_pred))


Akurasi: 0.8404
              precision    recall  f1-score   support

    negative       0.83      0.85      0.84     15707
    positive       0.85      0.83      0.84     15707

    accuracy                           0.84     31414
   macro avg       0.84      0.84      0.84     31414
weighted avg       0.84      0.84      0.84     31414



In [42]:
from sklearn.model_selection import GridSearchCV
param_grid = {
    'C': [0.01, 0.1, 1, 5, 10],
    'penalty': ['l2'],  # 'l1' butuh solver='liblinear'
}
grid = GridSearchCV(LogisticRegression(max_iter=40000), param_grid, cv=5)
grid.fit(X_train, y_train)
print("Best Params:", grid.best_params_)
# Evaluasi model
accuracy = accuracy_score(y_test, y_pred)
print(f"Akurasi: {accuracy:.4f}")
print(classification_report(y_test, y_pred))


Best Params: {'C': 1, 'penalty': 'l2'}
Akurasi: 0.8404
              precision    recall  f1-score   support

    negative       0.83      0.85      0.84     15707
    positive       0.85      0.83      0.84     15707

    accuracy                           0.84     31414
   macro avg       0.84      0.84      0.84     31414
weighted avg       0.84      0.84      0.84     31414



### Stochastic Gradient Descent

In [35]:
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import train_test_split


# Model: Stochastic Gradient Descent
sgd_model = SGDClassifier(random_state=42)
sgd_model.fit(X_train, y_train)

# Prediksi dan evaluasi
y_pred_sgd = sgd_model.predict(X_test)
print("SGD Classifier Model Evaluation:")
print("Accuracy:", accuracy_score(y_test, y_pred_sgd))
print(classification_report(y_test, y_pred_sgd))


SGD Classifier Model Evaluation:
Accuracy: 0.8327497294200038
              precision    recall  f1-score   support

    negative       0.81      0.87      0.84     15707
    positive       0.86      0.80      0.83     15707

    accuracy                           0.83     31414
   macro avg       0.83      0.83      0.83     31414
weighted avg       0.83      0.83      0.83     31414

